# Installation

이전 단계에서 Agent Framework를 설치하였습니다. 설치가 되지 않았을 경우 아래의 명령어를 터미널에서 실행하여 설치해주시기 바랍니다.

```
# Install core framework
pip install agent-framework
```

# Setup

Import required libraries for the notebook.

In [ ]:
import asyncio
import os
from typing import Annotated
from dotenv import load_dotenv

# Microsoft Agent Framework core
from agent_framework import ChatAgent, ChatMessage, TextContent, Role

# Azure integrations
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

# Tool decorators
from agent_framework import ai_function


load_dotenv(override=True)
print("✓ Libraries imported successfully")

환경 변수를 설정합니다. Microsoft Foundry 모델의 endpoint를 사용할 수 있습니다.

In [ ]:
# Check required environment variables
print("Checking environment variables...")

# For Azure OpenAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_version = os.getenv("AZURE_OPENAI_API_VERSION")

# For Azure AI Foundry
azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
azure_ai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if azure_openai_endpoint and azure_openai_deployment:
    print("✓ Azure OpenAI configuration found")
    SERVICE_TYPE = "azure_openai"
else:
    print("❌ Missing required environment variables!")
    print("\nFor Azure OpenAI, set:")
    print("  - AZURE_OPENAI_ENDPOINT")
    print("  - AZURE_OPENAI_API_KEY")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    print("\nOR for Azure AI Foundry, set:")
    print("  - AZURE_AI_PROJECT_ENDPOINT")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    
print(f"\nUsing service: {SERVICE_TYPE}")
azure_openai_deployment

# Case 1: 기본 Agent 생성과 실행

Microsoft Agent Framework에서는 에이전트 생성이 매우 간단합니다. Semantic Kernel처럼 Kernel 인스턴스를 요구하지 않고, 채팅 클라이언트에서 직접 에이전트를 생성할 수 있습니다.

## 🧪 Case 1.1: Azure OpenAI를 활용한 간단한 에이전트
---
AzureOpenAIChatClient를 사용하여 기본 ChatAgent를 생성합니다. 에이전트는 스레드 관리를 자동으로 처리합니다.

In [ ]:
# Create a chat client and agent with Azure OpenAI
azure_openai_chat_client = AzureOpenAIChatClient(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    api_version=azure_openai_version
)

# Create an agent - no Kernel needed!
agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that provides concise, informative answers.",
    name="SimpleAssistant"
)

# Run the agent with a simple question
async def run_simple_agent():
    result = await agent.run("마이크로소프트에 대해서 한글 3문장으로 얘기해줘.")
    print(f"Agent: {result.text}")

await run_simple_agent()

## 🧪 Case 1.2: 스트리밍 응답
---
Agent Framework은 실시간 출력을 위해 스트리밍 응답을 지원합니다.

In [ ]:
async def run_streaming_agent():
    print("Agent: ", end="", flush=True)
    async for update in agent.run_stream("Agentic AI를 3문장으로 쉽게 설명해줘."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming

await run_streaming_agent()

## 🧪 Case 1.3: 스레드를 활용한 다중 턴 (Multi-turn) 대화
---
에이전트는 스레드를 사용하여 대화의 맥락을 자동으로 유지합니다.

In [ ]:
async def multi_turn_conversation():
    
    # Create a new thread.
    thread = agent.get_new_thread()
    
    response1 = await agent.run("내 이름은 알렉스야. 나는 파이썬 프로그래맹을 좋아해.", thread=thread)
    print(f"Agent: {response1.text}\n")
    
    # Get the thread from the result
    serialized = await thread.serialize()

    # Later, deserialize and continue conversation
    new_thread = await agent.deserialize_thread(serialized)
    
    # Second message - uses the same thread to maintain context
    response2 = await agent.run("내 이름이 뭐지?", thread=new_thread)
    print(f"Agent: {response2.text}\n")
    
    # Third message - agent remembers both previous exchanges
    response3 = await agent.run("내가 좋아한다고 말한 프로그래밍 언어는 무엇인가요?", thread=new_thread)
    print(f"Agent: {response3.text}")

await multi_turn_conversation()

# Case 2: Custom Tools/Functions 활용한 Agents 
---
Agent Framework은 `@ai_function` decorator를 사용해 사용자 정의 함수를 손쉽게 확장할 수 있도록 지원합니다. 

* @ai_function 데코레이터는 에이전트가 호출할 수 있는 Custom Function을 간단한 파이썬 함수로 정의하고, 메타데이터(이름, 설명, 파라미터/반환 스키마 등)를 부여해 에이전트에 등록하는데 사용하는 데코레이터입니다. 별도의 플러그인 래퍼나 복잡한 클래스 상속 없이, 함수 정의만으로 에이전트의 능력을 확장할 수 있습니다.

### 🧪 Case 2.1: Azure Open AI Client with Custom Function Tools

In [ ]:
# Define custom functions using @ai_function decorator
@ai_function(description="Get the current weather for a location")
def get_weather(location: Annotated[str, "The city name"]) -> str:
    """Returns mock weather data for the given location."""
    # In a real scenario, this would call a weather API
    weather_data = {
        "Seoul": "Sunny, 22°C",
        "New York": "Cloudy, 18°C",
        "London": "Rainy, 15°C",
        "Tokyo": "Clear, 20°C"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

@ai_function(description="Get the current time in a timezone")
def get_time(timezone: Annotated[str, "Timezone (e.g., 'UTC', 'EST', 'KST')"]) -> str:
    """Returns mock time for the given timezone."""
    from datetime import datetime
    # Simplified for demo - in production use proper timezone handling
    return f"Current time in {timezone}: {datetime.now().strftime('%H:%M:%S')}"

# Create an agent with these tools
weather_agent = ChatAgent(
    chat_client=azure_openai_chat_client,
    instructions="You are a helpful assistant that can provide weather and time information.",
    name="WeatherAssistant",
    tools=[get_weather, get_time]
)

# Test the agent with tool calls
async def test_tools():
    result = await weather_agent.run("Seoul의 날씨는 어떤가요?")
    print(f"Agent: {result.text}\n")
    
    result2 = await weather_agent.run("KST 시간은 몇 시인가요?")
    print(f"Agent: {result2.text}")

await test_tools()

# Case 4: Azure AI Agent Client를 활용한 엔터프라이즈 도구
---
Microsoft Agent Framework는 Azure 서비스와 원할하게 통합되는 강력한 엔터프라이즈급 호스팅 도구를 제공합니다. 이러한 도구는 에이전트가 웹 검색, 문서 검색, 외부 시스템 연동과 같은 작업을 수행할 수 있도록 지원합니다.

## Hosted Tools 비교

| Tool | Purpose | Service | Key Features |
|------|---------|---------|--------------|
| **HostedFileSearchTool** | Document/Data Search | Azure AI Search | ✅ Index-based search<br>✅ Semantic search<br>✅ Configurable ranking |
| **HostedWebSearchTool** | Real-time Web Search | Bing Grounding | ✅ Current information<br>✅ Source citations<br>✅ Web grounding |
| **Local MCP** | Custom Integrations | Local Server | ✅ Full control<br>✅ Custom logic<br>✅ Local resources |
| **Hosted MCP** | External Services | Remote Server | ✅ Third-party APIs<br>✅ Scalable<br>✅ Managed infrastructure |

### [Case 4]를 실습하기 위한 사전 준비 사항:

Azure 계정에 로그인 합니다. (Azure CLI 로그인은 터미널에서 먼저 수행해주세요.)
- az login --use-device-code
- az account set --subscription "your-subscription-id"

### 현재 로그인 상태 및 subscription 확인
- az account show --output table

**For Azure AI Search (Case 4.1):**
- Azure AI Search 서비스를 위한 인덱싱된 데이터
- Azure AI Project의 Search connection 설정
- Index name (e.g., "hotels-sample-index")

**For Bing Grounding (Case 4.2):**
- Bing Grounding connection in Azure AI Project
- `BING_CONNECTION_NAME` or `BING_CONNECTION_ID` environment variable

**For MCP Tools (Case 4.3 & 4.4):**
- Model Context Protocol (MCP)에 대한 이해
- MCP 서버 설정 (local or hosted)

**본 교육에서는 Case 4.2 Bing Grounding을 진행합니다. 나머지 케이스에 대한 소스 코드는 [원본 GitHub 리포지토리의 파이썬 코드](https://github.com/Azure/agent-innovator-lab/blob/main/0_basic-agent/AgentFramework/1_basic-concept-with-msaf.ipynb)를 참조하시기 바랍니다.**

## 🧪 Case 4.2: HostedWebSearchTool를 활용한 Bing Grounding
---
실시간 웹 데이터를 활용해 질문에 답하기 위해 Bing Search API를 사용한 웹 기반 처리 방법을 보여줍니다.

**Setup:**
- Azure AI Project with Bing Search connection
- Environment variables: `AZURE_AI_PROJECT_ENDPOINT`, `BING_CONNECTION_NAME`

**Key Modules:**
- `HostedWebSearchTool` from `agent_framework.azure_ai.tools`
- Provides grounded responses with web search results and citations

In [ ]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from agent_framework import HostedWebSearchTool
from agent_framework import ChatAgent

azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
azure_ai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

async def bing_grounding_demo():
    """
    Demonstrates using Bing Search API for web grounding to answer questions with real-time web data.
    """
    
    try:
        # Get Azure AI Project endpoint
        bing_connection_id = os.environ.get("BING_GROUNDING_CONNECTION_ID")
        
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        # Create web search tool with Bing connection
        web_search_tool = HostedWebSearchTool(
            
            additional_properties={
                "connection_id": bing_connection_id,
                "top_k": 3,  # Get top 3 search results
            }
        )
        
        azure_ai_agent_client = AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=azure_project_endpoint,
            model_deployment_name=azure_ai_deployment
        )
        
        # Create agent with web search capability
        agent = ChatAgent(
            #chat_client=azure_openai_chat_client,
            chat_client=azure_ai_agent_client,
            name="Web Research Assistant",
            instructions="You are a helpful research assistant. Use web search to find current information and provide grounded answers with citations.",
            tools=[web_search_tool]
        )
        
        # Test questions that require web search
        USER_INPUTS = [
            "오늘의 날씨는 어떻습니까?"
        ]
        
        print("=== Azure AI Agent with Bing Grounding Search ===")
        print("This agent can search the web for current information and provide grounded answers.\n")
        
        # Simulate conversation with the agent
        for user_input in USER_INPUTS:
            print(f"User: {user_input}")
            print("Agent: ", end="", flush=True)
            
            # Stream the response for better user experience
            async for chunk in agent.run_stream(user_input):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            print("\n" + "=" * 50 + "\n")
        
        print("Web search conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        # Cleanup: Close Azure AI Agent client if it was created
        try:
            await azure_ai_agent_client.close()    
        except Exception as cleanup_error:
            print(f"Cleanup warning: {cleanup_error}")

await bing_grounding_demo()

## 🧪 Case 4.3: 로컬 MCP 서버 연동
---
로컬에서 실행 중인 Model Context Protocol(MCP) 서버에 연결해, 커스텀 도구로 에이전트 기능을 확장하는 방법을 보여줍니다.
### 설정:

- 로컬 MCP 서버 실행 (예: 파일 시스템 도구, 데이터베이스 커넥터 등)
- MCP 서버 URL 또는 설정 정보
- 환경 변수: AZURE_OPENAI_ENDPOINT

### 주요 모듈:

- MCP 서버에서 도구를 자동 탐색·등록
- MCP 규격을 준수하는 모든 서버 구현과 함께 동작

In [ ]:
import os
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from agent_framework import ChatAgent, MCPStreamableHTTPTool


async def local_mcp_demo():
    """
    Demonstrates connecting to a local Model Context Protocol (MCP) server 
    to extend agent capabilities with custom tools.
    
    This example shows how to use Microsoft Learn MCP server with Azure AI Agent.
    """
    
    try:
        # Create Azure AI Agent client with credential
        credential = AzureCliCredential()
        
        print("=== Azure AI Agent with Local MCP Server ===")
        print("This agent can use MCP tools to access Microsoft documentation.\n")
        
        # Connect to local MCP server and create agent with MCP tools
        async with (
            AzureAIAgentClient(
                async_credential=credential,
                project_endpoint=azure_project_endpoint,
                model_deployment_name=azure_ai_deployment
            ) as azure_ai_agent_client,
            MCPStreamableHTTPTool(
                name="Local Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
            ) as mcp_server
        ):
            # Create agent with MCP tools
            agent = ChatAgent(
                chat_client=azure_ai_agent_client,
                name="Local MCP Assistant(AzureAIAgentClient)",
                instructions="You are a helpful assistant that can help with Microsoft documentation questions using MCP tools.",
                tools=mcp_server,
                store=True
            )
            
            # Test questions that use MCP tools
            USER_INPUTS = [
                "How to create an Azure storage account using az cli? response with a simple answer and an example.",
                "What is Microsoft Agent Framework? response with key concepts and a main python code example."
            ]
            
            # Simulate conversation with the agent
            for user_input in USER_INPUTS:
                print(f"User: {user_input}")
                print("Agent: ", end="", flush=True)
                
                # Stream the response for better user experience
                async for chunk in agent.run_stream(user_input):
                    if chunk.text:
                        print(chunk.text, end="", flush=True)
                print("\n" + "=" * 50 + "\n")
            
            print("MCP tool conversation completed!")
    
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()

await local_mcp_demo()

## 📊 Case 4 비교: Enterprise Tools

| Feature | AI Search (4.1) | Bing Grounding (4.2) | Local MCP (4.3) | Hosted MCP (4.4) |
|---------|----------------|---------------------|----------------|------------------|
| **Tool Type** | `HostedFileSearchTool` | `HostedWebSearchTool` | `MCPClient` | `HostedMCPTool` |
| **Client Type** | `AzureAIAgentClient` | `AzureAIAgentClient` | `AzureOpenAIChatClient` | `AzureAIAgentClient` |
| **Data Source** | Azure AI Search vector stores | Bing Search API | Local MCP server process | Azure-hosted MCP server |
| **Setup Complexity** | Medium (requires vector store) | Low (requires Bing connection) | Medium (requires local server) | Low (requires Azure connection) |
| **Use Cases** | Internal document search, knowledge base | Real-time web information | Custom local tools, file system | Enterprise-grade custom tools |
| **Authentication** | `DefaultAzureCredential` | `DefaultAzureCredential` | `AzureCliCredential` | `DefaultAzureCredential` |
| **Grounding** | Citations from indexed docs | Web citations and sources | N/A (custom tool output) | Depends on MCP server |
| **Scalability** | High (Azure managed) | High (Azure managed) | Low (local process) | High (Azure managed) |
| **Customization** | Limited to indexed data | Limited to web search | Full (custom MCP tools) | Full (custom MCP tools) |
| **Cost** | AI Search + AI Project | Bing API + AI Project | Azure OpenAI only | MCP hosting + AI Project |

**Key Takeaways:**
- **AI Search (4.1)**: 자체 문서와 지식 기반을 의미 검색으로 탐색할 때 가장 적합
- **Bing Grounding (4.2)**: 출처가 명시된 실시간 웹 정보를 활용할 때 가장 적합
- **Local MCP (4.3)**: 로컬 머신에서 맞춤형 도구로 개발 및 테스트할 때 가장 적합
- **Hosted MCP (4.4)**: 엔터프라이즈 보안과 확장성을 갖춘 맞춤형 도구의 프로덕션 배포에 가장 적합